# Task Navigation Efficiency Evaluator

## Objective
This sample demonstrates how to use task navigation efficiency evaluator on agent data. The supported input formats include:
- simple data such as strings and `dict` describing task responses;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
This is a deterministic evaluator that compares navigation paths.

### Prerequisite
```bash
pip install azure-ai-evaluation
```
No environment variables are required for this evaluator.

The Task Navigation Efficiency evaluator assesses how efficiently an AI agent navigates through task completion by examining:
- The number of steps taken to complete the task
- Whether the agent took the most efficient path
- Comparison against ideal or ground truth navigation steps
- Unnecessary detours or redundant actions

The evaluator uses a scoring system that measures navigation efficiency:

    - Higher scores indicate more efficient navigation
    - Lower scores indicate inefficient paths with unnecessary steps
    - The evaluation considers the optimal path as the baseline

This evaluation focuses on measuring whether the agent takes an efficient route to task completion, minimizing unnecessary steps while still achieving the desired outcome.

Task Navigation Efficiency requires following input:
- Query - This can be a single query or a list of messages(conversation history with agent). The original task request from the user.
- Response - Response from Agent (or any GenAI App). This can be a single text response or a list of messages generated as part of Agent Response.
- Ground Truth - (Optional) The ideal or expected sequence of steps to complete the task efficiently.
- Tool Definitions - (Optional) Tool(s) definition used by Agent to answer the query. Providing tool definitions helps the evaluator understand available actions.


### Initialize Task Navigation Efficiency Evaluator


In [ ]:
import os
from azure.ai.evaluation._evaluators._task_navigation_efficiency import _TaskNavigationEfficiencyEvaluator, _TaskNavigationEfficiencyMatchingMode
from pprint import pprint

# Initialize with exact match mode
task_navigation_efficiency = _TaskNavigationEfficiencyEvaluator(
    matching_mode=_TaskNavigationEfficiencyMatchingMode.EXACT_MATCH
)

# Other examples:
# For in-order matching (allows extra steps but requires correct order)
# task_navigation_efficiency = _TaskNavigationEfficiencyEvaluator(matching_mode=TaskNavigationEfficiencyMatchingMode.IN_ORDER_MATCH)

# For any-order matching (most lenient - allows extra steps and different order)  
# task_navigation_efficiency = _TaskNavigationEfficiencyEvaluator(matching_mode=TaskNavigationEfficiencyMatchingMode.ANY_ORDER_MATCH)

# Or use defaults (exact match mode)
# task_navigation_efficiency = _TaskNavigationEfficiencyEvaluator()

### Samples

#### Evaluating Simple Task Navigation

In [ ]:
query = "How is the weather in Seattle?"
response = "I'll check the weather for you. The current weather in Seattle is partly cloudy with a temperature of 15°C (59°F)."

# Ground truth represents the ideal efficient path
ground_truth = ["fetch_weather(location='Seattle')", "return_weather_information"]

# Basic evaluation without tool definitions
result = task_navigation_efficiency(query=query, response=response, ground_truth=ground_truth)
pprint(result)

#### Task Navigation with Tool Context

In [ ]:
query = "How is the weather in Seattle?"
response = [
    {
        "role": "assistant",
        "content": "I'll fetch the weather information for Seattle."
    },
    {
        "role": "assistant", 
        "content": "The current weather in Seattle is partly cloudy with a temperature of 15°C."
    }
]

ground_truth = ["fetch_weather", "provide_weather_info"]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

result = task_navigation_efficiency(
    query=query, 
    response=response, 
    ground_truth=ground_truth,
    tool_definitions=tool_definitions
)
pprint(result)

#### Complex Task with Multiple Steps Navigation

In [ ]:
query = "Can you send me an email with weather information for Seattle?"
response = [
    {
        "createdAt": "2025-03-26T17:27:35Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I'll help you get the weather information for Seattle and send it via email.",
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:37Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Let me first fetch the current weather for Seattle.",
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:38Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Now I'll compose and send the email with the weather information.",
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:42Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I have successfully sent you an email with the weather information for Seattle. The current weather is partly cloudy with a temperature of 15°C.",
            }
        ],
    },
]

# Ground truth represents the most efficient navigation path
ground_truth = [
    "fetch_weather(location='Seattle')", 
    "compose_email(weather_data)", 
    "send_email()"
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = task_navigation_efficiency(
    query=query, 
    response=response, 
    ground_truth=ground_truth,
    tool_definitions=tool_definitions
)
pprint(result)

#### Query as Conversation History with Navigation Steps
The evaluator also supports query as a list of messages representing conversation history. This helps evaluate navigation efficiency in the context of a full conversation.

In [ ]:
# Query as conversation history instead of a single string
query_as_conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can fetch weather information and send emails."
    },
    {
        "role": "user", 
        "content": "Hi, I need to plan my day. Can you check the weather in Seattle?"
    },
    {
        "role": "user",
        "content": "Also, please send me an email summary of the weather so I can reference it later."
    }
]

response = [
    {
        "role": "assistant",
        "content": "I'll help you with both tasks. Let me start by checking the weather in Seattle."
    },
    {
        "role": "assistant",
        "content": "The weather in Seattle is currently 15°C and partly cloudy with light winds."
    },
    {
        "role": "assistant",
        "content": "Now I'll send you an email with this weather information."
    },
    {
        "role": "assistant",
        "content": "Done! I've sent you an email with the weather summary for Seattle. You should receive it shortly."
    }
]

ground_truth = [
    "fetch_weather(location='Seattle')",
    "send_email(weather_summary)"
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = task_navigation_efficiency(
    query=query_as_conversation, 
    response=response, 
    ground_truth=ground_truth,
    tool_definitions=tool_definitions
)
pprint(result)

#### Example of Inefficient Navigation

In [ ]:
query = "How is the weather in Seattle?"
inefficient_response = [
    {
        "role": "assistant",
        "content": "Let me think about this request."
    },
    {
        "role": "assistant",
        "content": "You want to know about the weather. Let me consider what information you need."
    },
    {
        "role": "assistant", 
        "content": "Seattle is a city in Washington state. Let me check the weather there."
    },
    {
        "role": "assistant",
        "content": "I'll now fetch the weather information."
    },
    {
        "role": "assistant",
        "content": "The weather in Seattle is currently 15°C and partly cloudy."
    }
]

ground_truth = ["fetch_weather(location='Seattle')", "return_weather_information"]

# This response shows inefficient navigation with unnecessary steps
result = task_navigation_efficiency(
    query=query, 
    response=inefficient_response, 
    ground_truth=ground_truth,
    tool_definitions=tool_definitions
)
pprint(result)

### Sample 4: Task Navigation with Exact Parameter Matching

This sample demonstrates using `ground_truth` as a Tuple with exact parameter matching. The Tuple format allows you to specify not only which tools should be called, but also what exact parameters should be passed to each tool.

In [ ]:
query = "Check the weather in Seattle and send me an email about it"

# Agent response with multiple tool calls
response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_1",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"}
            }
        ]
    },
    {
        "role": "tool",
        "tool_call_id": "call_weather_1", 
        "content": '{"temperature": "15°C", "condition": "partly cloudy"}'
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_email_1",
                "name": "send_email",
                "arguments": {
                    "recipient": "user@example.com",
                    "subject": "Seattle Weather Update",
                    "body": "The weather in Seattle is 15°C and partly cloudy."
                }
            }
        ]
    },
    {
        "role": "tool",
        "tool_call_id": "call_email_1",
        "content": '{"status": "sent", "message_id": "msg_123"}'
    },
    {
        "role": "assistant",
        "content": "I've checked the weather in Seattle (15°C, partly cloudy) and sent you an email with the details."
    }
]

# Ground truth as Tuple with exact parameter matching
# This format allows you to specify not just which tools should be called,
# but also what exact parameters should be passed to each tool
ground_truth = (
    ["fetch_weather", "send_email"],  # Expected tool sequence
    {
        "fetch_weather": {"location": "Seattle"},  # Exact expected parameters
        "send_email": {
            "recipient": "user@example.com",
            "subject": "Seattle Weather Update",
            "body": "The weather in Seattle is 15°C and partly cloudy."
        }
    }
)

result = task_navigation_efficiency(response=response, ground_truth=ground_truth)
pprint(result)